Library imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat

from Toolbox_GCSR.Preprocessing import Processor
from sklearn.preprocessing import minmax_scale
from Toolbox_GCSR.utility import eval_band, eval_band_cv
from Toolbox_GCSR.EGCSR_BS_Ranking import EGCSR_BS_Ranking

from Toolbox_bombs.runner import main
from Toolbox_bombs.utils import Arguments

import matlab.engine
eng = matlab.engine.start_matlab()

eng.addpath(eng.genpath("Toolbox_SVD"));
eng.addpath(eng.genpath("Toolbox_OCF"));

Data Importation and adequation

In [2]:
root = 'D:\Python\HSI_Files\\'
im_, gt_ = 'CornData', 'CornLabels'

hsi_path = {'img_path': root + im_ + '.mat', 'gt_path': root + gt_ + '.mat'}

#img_path = root + im_ + '.mat'
#gt_path = root + gt_ + '.mat'


Parameters and variables

In [3]:
#dictionary for subsets selected from all algorithms
bands_selected = {}

#The number of iterations and number of selecte bands
iter = 8
factor = 4
#each algorithm will select bands for each iter*factor number of bands

bands_matrix = np.zeros((iter,factor*iter))

Algoritmo SVD

In [4]:
def svd(hsi_path, band_num):

    data_m = eng.double(eng.importdata(hsi_path['img_path']))

    select_svd = eng.svd_function(data_m, band_num);

    bands = np.array(select_svd)

    bands = bands.reshape(1,band_num)

    bands = bands[0]

    bands = bands + 1
    
    return bands

Algoritmo OCF

In [5]:

def ocf(hsi_path, band_num):

    data_m = eng.double(eng.importdata(hsi_path['img_path']))

    select_ocf = eng.ocf_trc_fdpc(data_m, band_num);

    bands = np.array(select_ocf)

    bands = bands.reshape(1,band_num)

    bands = bands[0]

    bands = bands + 1

    return bands

Algoritmo bombs

In [6]:
RESULTS_DIR = os.path.join(
    os.getenv("Bombs_results", os.path.join("..", "..", "results")),
    "bombs_band_selection"
)

def bombs(hsi_path, bands_num):

    arguments = Arguments(
            bands_per_antibody=bands_num,
            data_path = hsi_path['img_path'],
            ref_map_path=hsi_path['gt_path'],
            dest_path=RESULTS_DIR,
            Gmax=6,
            Na=10,
            Nd=25,
            Nc=25,
            TD_size=30,
            P_init_size=100,
        )
    return main(args=arguments)

Algoritmo GCSR

In [7]:
#
def EGCSR(hsi_path, bands_num):

    p = Processor()
    hsi_data, _ = p.prepare_data(hsi_path['img_path'], hsi_path['gt_path'])

    alg_Ranking = EGCSR_BS_Ranking(bands_num, regu_coef=1e2, n_neighbors=3, ro=0.8)
    alg_Ranking.predict(hsi_data)
    return alg_Ranking.band_indx


processing functions

In [8]:
BS_functions = []
BS_functions.append(EGCSR)
BS_functions.append(bombs)
BS_functions.append(svd)
BS_functions.append(ocf)

for i in range(len(BS_functions)):

    bands_selected[BS_functions[i].__name__] = bands_matrix

band selectrion process

In [9]:
for alg in BS_functions:

    print(alg.__name__ + ' START')
    for i in range(iter):
        
        band_count = (i+1)*factor

        alg_bands = alg(hsi_path, band_count)

        bands_selected[alg.__name__][i,:len(alg_bands)] = alg_bands

    print(alg.__name__ + ' ENDS')

EGCSR START
band index: [ 892  890  891 1248]
band indexlength: 4
band index: [ 888  894  893  889  892  890  891 1248]
band indexlength: 8
band index: [ 886  896  887  895  888  894  893  889  892  890  891 1248]
band indexlength: 12
band index: [ 899  885  898  897  886  896  887  895  888  894  893  889  892  890
  891 1248]
band indexlength: 16
band index: [ 901  883  900  884  899  885  898  897  886  896  887  895  888  894
  893  889  892  890  891 1248]
band indexlength: 20
band index: [ 419  903  882  902  901  883  900  884  899  885  898  897  886  896
  887  895  888  894  893  889  892  890  891 1248]
band indexlength: 24
band index: [ 416  417  420  418  419  903  882  902  901  883  900  884  899  885
  898  897  886  896  887  895  888  894  893  889  892  890  891 1248]
band indexlength: 28
band index: [ 422  414  415  421  416  417  420  418  419  903  882  902  901  883
  900  884  899  885  898  897  886  896  887  895  888  894  893  889
  892  890  891 1248]
band 